<a href="https://colab.research.google.com/github/jessycalais/API_BigQuery_Python/blob/main/sql_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Objetivo:**
* Utilizar o método `.shift()` da biblioteca Pandas para realizar análise semelhante a obtida com a função de janela LAG() no SQL.

1. Conexão com a API do **BigQuery**

In [1]:
from google.colab import auth
from google.cloud import bigquery

auth.authenticate_user()
print('Authenticated')

Authenticated


2. A tabela utilizada neste projeto está disponível no seguinte conjunto de dados públicos do **Big Query**:

> `bigquery-public-data.covid19_ecdc.covid_19_geographic_distribution_worldwide`

Antes de prosseguir, vamos realizar a conexão com um projeto privado no meu perfil de usuário:

In [2]:
client = bigquery.Client(project='formal-chassis-396823')

3.1. Consulta utilizando a instrução CASE e a função de janela LAG():

In [3]:
consulta_avancada_sql = client.query('''
                            SELECT
                              countries_and_territories,
                              CASE month
                                WHEN 1 THEN 'jan'
                                WHEN 2 THEN 'fev'
                                WHEN 3 THEN 'mar'
                                WHEN 4 THEN 'abr'
                                WHEN 5 THEN 'mai'
                                WHEN 6 THEN 'jun'
                                WHEN 7 THEN 'jul'
                                WHEN 8 THEN 'ago'
                                WHEN 9 THEN 'set'
                                WHEN 10 THEN 'out'
                                WHEN 11 THEN 'nov'
                                WHEN 12 THEN 'dez'
                              END AS mes,
                              SUM(daily_confirmed_cases) AS total_confirmed_cases,
                              LAG(SUM(daily_confirmed_cases))
                                OVER(PARTITION BY countries_and_territories ORDER BY month)
                                AS total_cases_previous_month,
                              ROUND((SAFE_DIVIDE(SUM(daily_confirmed_cases),
                                  (LAG(SUM(daily_confirmed_cases))
                                  OVER(PARTITION BY countries_and_territories ORDER BY month))) - 1) * 100, 2)
                                AS percentage_increase
                            FROM
                              `bigquery-public-data.covid19_ecdc.covid_19_geographic_distribution_worldwide`
                            WHERE
                              year = 2020
                              AND countries_and_territories = 'Brazil'
                            GROUP BY
                              month,
                              countries_and_territories
                            ORDER BY
                              countries_and_territories,
                              month,
                              total_confirmed_cases
                        ''').to_dataframe()

In [4]:
consulta_avancada_sql

,countries_and_territories,mes,total_confirmed_cases,total_cases_previous_month,percentage_increase
0,Brazil,jan,0,<NA>,NaN
1,Brazil,fev,1,0,NaN
2,Brazil,mar,4578,1,457700.00
3,Brazil,abr,73583,4578,1507.32
4,Brazil,mai,420278,73583,471.16
5,Brazil,jun,869755,420278,106.95
6,Brazil,jul,1241907,869755,42.79
7,Brazil,ago,1252209,1241907,0.83
8,Brazil,set,915211,1252209,-26.91
9,Brazil,out,739136,915211,-19.24


3.2. Consulta simplificada sem utilizar CASE e LAG() para poder realizar os passos com o Pandas

In [5]:
consulta_simplificada_sql = client.query('''
                            SELECT
                              countries_and_territories,
                              month,
                              SUM(daily_confirmed_cases) AS total_confirmed_cases
                            FROM
                              `bigquery-public-data.covid19_ecdc.covid_19_geographic_distribution_worldwide`
                            WHERE
                              year = 2020
                              AND countries_and_territories = 'Brazil'
                            GROUP BY
                              month,
                              countries_and_territories
                            ORDER BY
                              countries_and_territories,
                              month,
                              total_confirmed_cases
                            ''').to_dataframe()

In [6]:
consulta_simplificada_sql

,countries_and_territories,month,total_confirmed_cases
0,Brazil,1,0
1,Brazil,2,1
2,Brazil,3,4578
3,Brazil,4,73583
4,Brazil,5,420278
5,Brazil,6,869755
6,Brazil,7,1241907
7,Brazil,8,1252209
8,Brazil,9,915211
9,Brazil,10,739136


4. Utilizando as bibliotecas Pandas para renomear o nome dos meses do DataFrame `consulta_simplificada_sql`:

In [7]:
consulta_simplificada_sql['month'] = ['jan', 'fev', 'mar', 'abr', 'mai', 'jun', 'jul', 'ago', 'set', 'out', 'nov', 'dez']
consulta_simplificada_sql

,countries_and_territories,month,total_confirmed_cases
0,Brazil,jan,0
1,Brazil,fev,1
2,Brazil,mar,4578
3,Brazil,abr,73583
4,Brazil,mai,420278
5,Brazil,jun,869755
6,Brazil,jul,1241907
7,Brazil,ago,1252209
8,Brazil,set,915211
9,Brazil,out,739136


5. Criando a coluna `total_cases_previous_month` no DataFrame `consulta_simplificada_sql` utilizando o método `.shift()`:

In [8]:
consulta_simplificada_sql['total_cases_previous_month'] = consulta_simplificada_sql['total_confirmed_cases'].shift(periods=1,
                                                                                                                   axis=0)
consulta_simplificada_sql

,countries_and_territories,month,total_confirmed_cases,total_cases_previous_month
0,Brazil,jan,0,<NA>
1,Brazil,fev,1,0
2,Brazil,mar,4578,1
3,Brazil,abr,73583,4578
4,Brazil,mai,420278,73583
5,Brazil,jun,869755,420278
6,Brazil,jul,1241907,869755
7,Brazil,ago,1252209,1241907
8,Brazil,set,915211,1252209
9,Brazil,out,739136,915211


6. Criando a coluna `percentual_increase` no DataFrame `consulta_simplificada_sql`:

In [9]:
consulta_simplificada_sql['percent_increase'] = round(((consulta_simplificada_sql['total_confirmed_cases']
                                                      / consulta_simplificada_sql['total_cases_previous_month']) - 1) * 100,
                                                      2)
consulta_simplificada_sql

,countries_and_territories,month,total_confirmed_cases,total_cases_previous_month,percent_increase
0,Brazil,jan,0,<NA>,<NA>
1,Brazil,fev,1,0,inf
2,Brazil,mar,4578,1,457700.0
3,Brazil,abr,73583,4578,1507.32
4,Brazil,mai,420278,73583,471.16
5,Brazil,jun,869755,420278,106.95
6,Brazil,jul,1241907,869755,42.79
7,Brazil,ago,1252209,1241907,0.83
8,Brazil,set,915211,1252209,-26.91
9,Brazil,out,739136,915211,-19.24


7. Extra:
  * Formatação condicional;
  * Formatação da representação decimal.

In [10]:
consulta_simplificada_sql.style.background_gradient(axis=0,
                                                    gmap=consulta_simplificada_sql['total_confirmed_cases'],
                                                    cmap='coolwarm'
                                                    ) \
                                .format(decimal=',', precision=2)

,countries_and_territories,month,total_confirmed_cases,total_cases_previous_month,percent_increase
0,Brazil,jan,0,,
1,Brazil,fev,1,0,inf
2,Brazil,mar,4578,1,"457700,00"
3,Brazil,abr,73583,4578,"1507,32"
4,Brazil,mai,420278,73583,"471,16"
5,Brazil,jun,869755,420278,"106,95"
6,Brazil,jul,1241907,869755,"42,79"
7,Brazil,ago,1252209,1241907,"0,83"
8,Brazil,set,915211,1252209,"-26,91"
9,Brazil,out,739136,915211,"-19,24"
